In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import matplotlib.pyplot as plt

In [3]:
data_path = "/kaggle/input/gtzan-dataset-music-genre-classification/Data/features_3_sec.csv"
df_main = pd.read_csv(data_path)

In [4]:
df_main.shape
X = df_main.drop('label',axis=1)
y = df_main['label']

In [5]:
#encoding the labels for future referencing
index_map = ['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']
genre_map = {'blues':0,
             'classical':1,
            'country': 2,
            'disco': 3,
            'hiphop':4,
            'jazz':5,
            'metal':6,
            'pop':7,
            'reggae':8,
            'rock':9}


for i in range(len(y)):
    y[i] = genre_map[y[i]]
    
y = y.astype(int)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
X = X.drop(['filename','length','harmony_mean','harmony_var','perceptr_mean','perceptr_var'],axis = 1)
X.shape

(9990, 53)

# Train Test Split

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_rem, y_train, y_rem = train_test_split(X,y,stratify = y, test_size = 0.15)
X_test, X_val, y_test, y_val = train_test_split(X_rem, y_rem , stratify = y_rem , test_size = 0.5)
print(X_train.shape )
print(X_val.shape)

(8491, 53)
(750, 53)


In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)
X_val = sc.transform(X_val)

# SVM Classifier

In [9]:
from sklearn.svm import SVC
SVM_classifier = SVC(kernel = 'poly',C =0.1,gamma = 1)
SVM_classifier.fit(X_train,y_train)

SVC(C=0.1, gamma=1, kernel='poly')

In [10]:
y_pred = SVM_classifier.predict(X_val)
y_pred

array([6, 6, 9, 4, 4, 4, 5, 8, 7, 8, 6, 3, 2, 8, 0, 9, 8, 5, 1, 0, 7, 7,
       2, 0, 6, 1, 3, 9, 1, 7, 0, 4, 8, 8, 7, 0, 0, 8, 8, 6, 3, 5, 0, 1,
       2, 5, 4, 5, 1, 1, 1, 1, 4, 3, 8, 0, 0, 7, 1, 0, 1, 0, 4, 9, 0, 6,
       5, 7, 8, 2, 2, 5, 6, 1, 0, 6, 1, 0, 1, 1, 0, 1, 1, 2, 3, 8, 1, 6,
       9, 5, 6, 2, 8, 6, 5, 6, 6, 5, 5, 9, 9, 1, 4, 4, 5, 6, 9, 0, 6, 5,
       1, 0, 1, 3, 2, 6, 9, 0, 4, 5, 1, 0, 0, 1, 1, 2, 1, 5, 7, 4, 3, 8,
       6, 8, 7, 6, 2, 2, 2, 4, 9, 4, 8, 8, 0, 0, 0, 1, 7, 4, 4, 5, 5, 4,
       8, 3, 5, 2, 9, 8, 1, 0, 1, 7, 9, 3, 4, 7, 2, 3, 8, 5, 1, 0, 3, 0,
       3, 7, 8, 0, 8, 2, 1, 0, 7, 5, 8, 5, 2, 4, 1, 7, 0, 1, 4, 8, 0, 4,
       8, 4, 2, 3, 4, 4, 7, 7, 1, 9, 1, 3, 0, 0, 8, 5, 3, 5, 8, 3, 5, 9,
       3, 6, 5, 1, 4, 2, 9, 6, 7, 4, 2, 7, 2, 5, 7, 7, 4, 6, 9, 2, 6, 9,
       6, 8, 5, 4, 5, 3, 8, 6, 5, 8, 6, 9, 7, 1, 4, 1, 1, 9, 4, 3, 2, 3,
       8, 3, 8, 8, 2, 7, 8, 8, 4, 9, 5, 0, 3, 0, 4, 4, 3, 2, 4, 5, 1, 4,
       4, 9, 8, 2, 2, 6, 9, 3, 4, 7, 7, 0, 5, 5, 7,

In [11]:
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score
cm = confusion_matrix(y_val,y_pred)
cm


print('accuracy score: ', accuracy_score(y_val,y_pred))
print('f1_score : ', f1_score(y_val,y_pred,average='micro'))


accuracy score:  0.8813333333333333
f1_score :  0.8813333333333333


# GridSearchCV

In [12]:
from sklearn.model_selection import GridSearchCV

#defining parameter range
param_grid = {'C' : [0.1,1,10],
             'gamma': [1, 0.1],
             'kernel': ['rbf','poly']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)


history_svm = grid.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.159 total time=   7.5s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.150 total time=   7.5s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.167 total time=   7.6s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.153 total time=   7.5s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.152 total time=   7.5s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.867 total time=   2.2s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.858 total time=   2.2s
[CV 3/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.859 total time=   2.3s
[CV 4/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.870 total time=   2.3s
[CV 5/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.862 total time=   2.3s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.469 total time=   6.8s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;

In [13]:
# print best parameter after tuning
print(grid.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'C': 0.1, 'gamma': 0.1, 'kernel': 'poly'}
SVC(C=0.1, gamma=0.1, kernel='poly')


In [14]:
grid_predictions = grid.predict(X_val)
print('accuracy score: ', accuracy_score(y_val,grid_predictions))
print('f1_score : ', f1_score(y_val,grid_predictions,average='micro'))


accuracy score:  0.884
f1_score :  0.884


# Metadata Generator

In [15]:
import librosa
def getdataf(filename):
    


    y, sr = librosa.load(filename)
    #fetching tempo

    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)

    #fetching beats

    y_harmonic, y_percussive = librosa.effects.hpss(y)
    tempo, beat_frames = librosa.beat.beat_track(y=y_percussive,sr=sr)

    #chroma_stft

    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)

    #rmse

    rmse = librosa.feature.rms(y=y)

    #fetching spectral centroid

    spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]

    #spectral bandwidth

    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)

    #fetching spectral rolloff

    spec_rolloff = librosa.feature.spectral_rolloff(y=y+0.01, sr=sr)[0]

    #zero crossing rate

    zero_crossing = librosa.feature.zero_crossing_rate(y)
#     harmonics = [1./3, 1./2, 1, 2, 3, 4]
#     S = np.abs(librosa.stft(y))
#     fft_freqs = librosa.fft_frequencies(sr=sr)
#     S_harm = librosa.interp_harmonics(S, freqs=fft_freqs, harmonics=harmonics, axis=0)
#     S = np.abs(librosa.stft(y))
#     fft_freqs = librosa.fft_frequencies(sr=sr)
#     t_harmonics = librosa.interp_harmonics(S, freqs=fft_freqs, harmonics=harmonics, axis=0)
    
#     C = np.abs(librosa.cqt(y, sr=sr, fmin=librosa.note_to_hz('A1')))
#     freqs = librosa.cqt_frequencies(C.shape[0],
#                                 fmin=librosa.note_to_hz('A1'))
#     perceptual_CQT = librosa.perceptual_weighting(C**2,
#                                               freqs,
#                                               ref=np.max)
    #mfcc

    mfcc = librosa.feature.mfcc(y=y, sr=sr)

    #metadata dictionary

    metadata_dict = {'chroma_stft_mean':np.mean(chroma_stft),'chroma_stft_var':np.var(chroma_stft),'rmse_mean':np.mean(rmse),'rmse_var':np.var(rmse),
                     'spectral_centroid_mean':np.mean(spec_centroid),'spectral_centroid_var':np.var(spec_centroid),'spectral_bandwidth_mean':np.mean(spec_bw),'spectral_bandwidth':np.var(spec_bw), 
                     'rolloff_mean':np.mean(spec_rolloff),'rolloff_var':np.var(spec_rolloff), 'zero_crossing_rate_mean':np.mean(zero_crossing),'zero_crossing_rate_var':np.var(zero_crossing),
                    # 'harmony_mean':np.mean(t_harmonics),'harmony_var':np.var(t_harmonics),
                    # 'perceptr_mean':np.mean(perceptual_CQT),'perceptr_var':np.var(perceptual_CQT),
                     'tempo':tempo}

    for i in range(1,21):
        metadata_dict.update({'mfcc'+str(i)+'_mean':np.mean(mfcc[i-1])})
        metadata_dict.update({'mfcc'+str(i)+'_var':np.var(mfcc[i-1])})
    df_meta = pd.DataFrame(metadata_dict,index=[0])
    return df_meta


In [16]:
#Prediction Accuracy after hyperparameter tuning
grid_predictions_test = grid.predict(X_test)
print('accuracy score: ', accuracy_score(y_test,grid_predictions_test))
print('f1_score : ', f1_score(y_test,grid_predictions_test,average='micro'))


accuracy score:  0.8865153538050734
f1_score :  0.8865153538050734


# XGBoost

In [17]:
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt

In [18]:
def plotTraining(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, 'bo', label='Training_acc')
    plt.plot(epochs, val_acc, 'b', label='Validation_acc')
    plt.title('Training and validation accuracy')
    plt.legend() 

    plt.figure() 

    plt.plot(epochs, loss, 'bo', label='Training_loss')
    plt.plot(epochs, val_loss, 'b', label='Validation_loss')
    plt.title('Training and validation loss')
    plt.legend() 

    plt.show()

In [19]:
model_xgboost = xgboost.XGBClassifier(learning_rate=0.1,
                                      max_depth=5,
                                      n_estimators=5000,
                                      subsample=0.5,
                                      colsample_bytree=0.5,
                                      eval_metric='auc',
                                      verbosity=1)

eval_set = [(X_val, y_val)]

model_xgboost.fit(X_train,
                  y_train,
                  early_stopping_rounds=10,
                  eval_set=eval_set,
                  verbose=True)

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[0]	validation_0-auc:0.88650
[1]	validation_0-auc:0.91704
[2]	validation_0-auc:0.92627
[3]	validation_0-auc:0.93770
[4]	validation_0-auc:0.94136
[5]	validation_0-auc:0.94440
[6]	validation_0-auc:0.94895
[7]	validation_0-auc:0.95197
[8]	validation_0-auc:0.95263
[9]	validation_0-auc:0.95427
[10]	validation_0-auc:0.95628
[11]	validation_0-auc:0.95743
[12]	validation_0-auc:0.95854
[13]	validation_0-auc:0.95992
[14]	validation_0-auc:0.96097
[15]	validation_0-auc:0.96136
[16]	validation_0-auc:0.96236
[17]	validation_0-auc:0.96271
[18]	validation_0-auc:0.96403
[19]	validation_0-auc:0.96461
[20]	validation_0-auc:0.96549
[21]	validation_0-auc:0.96596
[22]	validation_0-auc:0.96629
[23]	validation_0-auc:0.96668
[24]	validation_0-auc:0.96703
[25]	validation_0-auc:0.96739
[26]	validation_0-auc:0.96790
[27]	validation_0-auc:0.96860
[28]	validation_0-auc:0.96905
[29]	validation_0-auc:0.96929
[30]	validation_0-auc:0.96957
[31]	validation_0-auc:0.97001
[32]	validation_0-auc:0.97047
[33]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=5000,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [20]:
# test_predict1 = model_xgboost.predict(df)
# print(test_predict1)

# Testing on New song

In [21]:
#df genration for new song
df=getdataf("/kaggle/input/rocktestwav/hip-hop-rock-beats-118000.wav")
df = sc.transform(df)
print(df)
test_predict = grid.predict(df)
print(test_predict)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  # Remove the CWD from sys.path while we load stuff.


[[ 0.82308565 -0.34598739  1.66260441  2.9056037   0.22782498  0.48617129
   0.28953305  2.17392017  0.18830958  1.05858587  0.0963586   0.21974676
  -0.77614953  1.08089735  1.43490786 -0.25804529  2.73157977 -0.41628362
   0.50982009 -1.75710732  3.34543994 -0.06572728  0.4526971  -1.17042962
   1.13385898  0.02311642  0.65680581 -1.17004078  1.26770779  0.56892835
   1.03946812 -0.75812081  0.78934311  0.10020593  0.23479899 -0.56463604
   0.40182921  0.02736539  0.27424579 -1.21516586  0.41442383 -0.20809984
   0.36229313 -0.59876927  0.54937956 -0.31791944  0.10622423 -0.22112392
  -0.12132944  0.2327117  -0.36571973  0.4904691  -0.26903607]]
[4]


/opt/conda/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- rmse_mean
- rmse_var
- spectral_bandwidth
Feature names seen at fit time, yet now missing:
- rms_mean
- rms_var
- spectral_bandwidth_var

  warnings.warn(message, FutureWarning)


In [22]:
test_predict1 = model_xgboost.predict(df)
print(test_predict1)

[4]


# Stacking the Models

In [23]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

estimator_list = [
    ('model_xgboost',model_xgboost),
    ('grid',grid)
]

#Build Stack Model

StackModel = StackingClassifier(estimators = estimator_list,final_estimator = LogisticRegression())

StackModel.fit(X_train,y_train)

#Prediction by Ensembled Model
Stack_model_pred = StackModel.predict(X_val)
print('accuracy score: ', accuracy_score(y_val,Stack_model_pred))
print('f1_score : ', f1_score(y_val,Stack_model_pred,average='micro'))

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.159 total time=   7.4s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.150 total time=   7.3s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.167 total time=   7.3s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.153 total time=   7.3s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.152 total time=   7.3s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.867 total time=   2.2s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.858 total time=   2.2s
[CV 3/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.859 total time=   2.2s
[CV 4/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.870 total time=   2.2s
[CV 5/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.862 total time=   2.2s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.469 total time=   6.6s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


accuracy score:  0.9026666666666666


ValueError: Found input variables with inconsistent numbers of samples: [749, 750]

In [24]:
print('accuracy score: ', accuracy_score(y_val,Stack_model_pred))
print('f1_score : ', f1_score(y_val,Stack_model_pred,average='micro'))

accuracy score:  0.9026666666666666
f1_score :  0.9026666666666666


In [25]:
# print best parameter after tuning
print(grid.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'C': 0.1, 'gamma': 0.1, 'kernel': 'poly'}
SVC(C=0.1, gamma=0.1, kernel='poly')


In [26]:
print(StackModel.predict(df))

[4]


In [27]:
#Saving the model
from joblib import Parallel, delayed
import joblib
  
  
#Save the Model using pickle
joblib.dump(StackModel, 'final.pkl')
  
# Load the model from the file
Model_from_joblib = joblib.load('final.pkl')
  
# Use the loaded model to make predictions
Model_from_joblib.predict(X_val)

array([6, 6, 9, 4, 4, 4, 5, 4, 7, 8, 6, 3, 2, 8, 0, 9, 8, 5, 1, 0, 7, 7,
       2, 0, 6, 1, 3, 9, 5, 7, 0, 4, 8, 8, 7, 6, 0, 8, 8, 6, 3, 5, 0, 1,
       2, 5, 4, 5, 1, 1, 1, 1, 4, 3, 8, 0, 0, 7, 1, 0, 1, 0, 4, 9, 2, 6,
       5, 7, 8, 2, 2, 5, 6, 1, 0, 6, 1, 0, 1, 1, 0, 1, 1, 2, 7, 8, 1, 9,
       9, 5, 6, 2, 8, 6, 5, 6, 6, 5, 5, 9, 9, 1, 4, 4, 5, 6, 9, 0, 6, 5,
       1, 0, 1, 3, 2, 6, 9, 0, 4, 5, 1, 0, 0, 1, 1, 6, 1, 5, 7, 4, 3, 8,
       6, 8, 7, 6, 2, 2, 9, 4, 6, 4, 8, 8, 0, 0, 0, 1, 3, 7, 4, 5, 5, 4,
       8, 3, 5, 2, 9, 8, 1, 0, 1, 7, 9, 3, 4, 7, 2, 9, 8, 5, 1, 0, 3, 0,
       3, 3, 2, 0, 8, 2, 1, 0, 7, 5, 9, 5, 2, 4, 1, 7, 0, 1, 4, 8, 0, 4,
       8, 4, 9, 3, 4, 4, 7, 7, 1, 9, 1, 3, 0, 2, 8, 5, 9, 5, 8, 3, 0, 3,
       3, 6, 5, 1, 4, 3, 9, 2, 7, 4, 2, 7, 2, 5, 7, 7, 4, 6, 9, 2, 6, 9,
       6, 8, 5, 4, 5, 3, 8, 6, 5, 8, 6, 9, 7, 1, 4, 1, 1, 9, 4, 3, 2, 3,
       8, 3, 8, 4, 2, 7, 2, 8, 4, 9, 5, 0, 3, 2, 4, 4, 3, 2, 4, 5, 1, 4,
       4, 9, 8, 2, 2, 6, 9, 3, 4, 7, 7, 0, 5, 5, 7,